In [1]:
from utils import *
import carla
from carla_control import carla_client
from utils.mods.post_processing import mono_label_distance_tracker, multi_classes_assemble_tracker
import numpy as np
import open3d as o3d
import torch
import time

from IPython.display import clear_output, display

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
logger = create_logger()

In [3]:
# client = carla_client(host = "192.168.31.13")
client = carla_client()

In [4]:
client.init_client()
client.start_client()

In [5]:
client.debug_luanch_test()

CFG_FILE = ".\\cfgs\\kitti_models\\pointpillar.yaml"
cfg = cfg_from_yaml_file(CFG_FILE, cfg)

pcs_dataset = carla_point_cloud_dataset(dataset_cfg = cfg.DATA_CONFIG, logger=logger, lidar=client.lidar_t, class_names=cfg.CLASS_NAMES)
model = torch.jit.load("./torch_scripts/point_pillar_model.pt")
object_tracker = multi_classes_assemble_tracker(num_classes=3, track_length=10)
model.cuda()
model.eval()

RecursiveScriptModule(
  original_name=PointPillar
  (vfe): RecursiveScriptModule(
    original_name=PillarVFE
    (pfn_layers): RecursiveScriptModule(
      original_name=ModuleList
      (0): RecursiveScriptModule(
        original_name=PFNLayer
        (linear): RecursiveScriptModule(original_name=Linear)
        (norm): RecursiveScriptModule(original_name=BatchNorm1d)
      )
    )
  )
  (map_to_bev_module): RecursiveScriptModule(original_name=PointPillarScatter)
  (backbone_2d): RecursiveScriptModule(
    original_name=BaseBEVBackbone
    (blocks): RecursiveScriptModule(
      original_name=ModuleList
      (0): RecursiveScriptModule(
        original_name=Sequential
        (0): RecursiveScriptModule(original_name=ZeroPad2d)
        (1): RecursiveScriptModule(original_name=Conv2d)
        (2): RecursiveScriptModule(original_name=BatchNorm2d)
        (3): RecursiveScriptModule(original_name=ReLU)
        (4): RecursiveScriptModule(original_name=Conv2d)
        (5): RecursiveScript

In [6]:
def carla_ticking(idx, data_dict): 
    before_time = time.perf_counter()
    
    client.carla_world.tick()
    spectator = client.carla_world.get_spectator()
    transform = client.vehicle.get_transform()
    spectator.set_transform(carla.Transform(transform.location + carla.Location(z=30),
                                                carla.Rotation(pitch=-90)))
    after_time = time.perf_counter()
    return after_time - before_time, None
    
def model_forwarding(idx, data_dict):
    final_boxes, final_scores, final_labels = None, None, None
    before_time = time.perf_counter()
    if data_dict["points"] is not None:
        data_dict = pcs_dataset.collate_batch([data_dict])
        load_data_to_gpu(data_dict)

        pred_dicts, _ = model.forward(data_dict)
        
        cls_preds = pred_dicts["pred_scores"]
        box_preds = pred_dicts["pred_boxes"]
        label_preds = pred_dicts["pred_labels"]

        selected, selected_scores = class_agnostic_nms(
                            box_scores=cls_preds, box_preds=box_preds,
                            score_thresh=0.3
                        )

        final_scores = selected_scores
        final_labels = label_preds[selected]
        final_boxes = box_preds[selected]
    after_time = time.perf_counter()
    return after_time - before_time, data_dict, final_boxes, final_scores, final_labels

def scene_rendering(idx, data_dict, vis, final_boxes, final_scores, final_labels):
    before_time = time.perf_counter()
    if data_dict["points"] is not None:
        draw_scenes(vis,
                    points=data_dict['points'][:, 1:], ref_boxes=final_boxes,
                    ref_scores=final_scores, ref_labels=final_labels, confidence=[0.0, 0.5, 0.0, 0.0]
                )

    vis.poll_events()
    vis.update_renderer()
    vis.clear_geometries()
    after_time = time.perf_counter()
    return after_time - before_time, None

In [7]:
vis = o3d.visualization.Visualizer()
vis.create_window()

with torch.no_grad():
    for idx, data_dict in enumerate(pcs_dataset):
        
        pre_time = data_dict["pre_time"]
        
        ticking_time, _ = carla_ticking(idx, data_dict)
        
        if data_dict["points"] is not None:
        
            forward_time, data_dict, final_boxes, final_scores, final_labels = model_forwarding(idx, data_dict)

            # render_time, _ = scene_rendering(idx, data_dict, vis, final_boxes, final_scores, final_labels)
            
            render_time = time.perf_counter()
            
            if final_boxes is not None:
                object_tracker.updates_object(final_boxes[np.newaxis, :, :], final_labels[np.newaxis, :], final_scores[np.newaxis, :])

            bounding_boxes, box_scores, box_labels, tracks = object_tracker.get_all_object()

            draw_scenes(vis,
                points=data_dict['points'][:, 1:], ref_boxes=bounding_boxes,
                ref_scores=box_scores, ref_labels=box_labels, confidence=None, tracks=tracks
            )
            vis.poll_events()
            vis.update_renderer()
            vis.clear_geometries()
            
            render_time = time.perf_counter() - render_time
        
            clear_output(wait=True)
            logger.info(f"Compute time: {pre_time:.3f} + {ticking_time:.3f} + {forward_time:.3f} + {render_time:.3f} == {pre_time + ticking_time + forward_time + render_time:.3f}s")
            logger.info(f"Target amount: {len(final_boxes if (final_boxes is not None) else [])}")
            logger.info(f"current uuid:{object_tracker.get_last_uuid()}")
            


2023-08-25 13:08:11,156   INFO  Compute time: 0.005 + 0.067 + 0.027 + 0.086 == 0.185s
2023-08-25 13:08:11,156   INFO  Target amount: 18
2023-08-25 13:08:11,157   INFO  current uuid:7366


RuntimeError: time-out of 10000ms while waiting for the simulator, make sure the simulator is ready and connected to 127.0.0.1:2000

(124788, 4)
(57300, 4)

{
	"class_name" : "ViewTrajectory",
	"interval" : 29,
	"is_loop" : false,
	"trajectory" : 
	[
		{
			"boundingbox_max" : [ 69.118263244628906, 39.679920196533203, 16.415634155273438 ],
			"boundingbox_min" : [ -0.059999999999999998, -39.679874420166016, -6.9146575927734375 ],
			"field_of_view" : 60.0,
			"front" : [ -0.90307097537632919, 0.0017988087570628851, 0.42948757574567964 ],
			"lookat" : [ 34.529131622314452, 2.288818359375e-05, 4.75048828125 ],
			"up" : [ 0.42948904059539766, 0.0070563614983622357, 0.90304450154510629 ],
			"zoom" : 0.69999999999999996
		}
	],
	"version_major" : 1,
	"version_minor" : 0
}